In [418]:
import pandas as pd
import pyreadr
import numpy as np
from tqdm import tqdm

from unidecode import unidecode

In [419]:
def write_error(msg):
    with open('errors.txt', 'w') as f:
        f.write(msg)

In [420]:
path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito.Rdata'
pathvote = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito_vote.Rdata'

df = pyreadr.read_r(path)['cand']
vote1 = pyreadr.read_r(pathvote)['cand']

#vote1 = vote1[vote1.tipo_eleicao == 'eleicao ordinaria']

vote = vote1

In [421]:
#Only result for 2802601 in 2004 is nao eleito handled
#Only result for 2803609 in 2004 is nao eleito handled

#Only result for 5203500 in 2012 is nao eleito Bom de Jesus
#Only result for 2803609 in 2016 is nao eleito Laranjeiras
#Only result for 3545506 in 2016 is nao eleito Sandovalina

In [422]:
vote.loc[len(vote.index)] = [2004, 1, 'eleicao ordinaria', 'SE','2802601','31518','prefeito', 'PP','11','88',np.nan, 'eleito', 2065]
vote.loc[len(vote.index)] = [2004, 1, 'eleicao ordinaria', 'SE', '2803609', '31712', 'prefeito', 'PTB', '14', '14', '119425', 'nao eleito', 5530]
vote.loc[len(vote.index)] = [2004, 1, 'eleicao ordinaria', 'SE', '2803609', '31712', 'prefeito', 'PFL', '25', '93', '1325512', 'eleito', 6609]

In [423]:
#vote[(vote.id_municipio == '2803609') & (vote.ano == 2016)]
#df[(df.ano == 2016) & (df.id_municipio == '2803609')]

In [424]:
df.shape

(100307, 27)

In [425]:
vote.loc[(vote.ano == 1996) | (vote.ano == 2000), 'sequencial_candidato'] = vote.loc[(vote.ano == 1996) | (vote.ano == 2000), 'sequencial_candidato'].fillna(vote.loc[(vote.ano == 1996) | (vote.ano == 2000), 'numero_candidato'])

df.loc[(df.ano == 1996) | (df.ano == 2000), 'sequencial'] = df.loc[(df.ano == 1996) | (df.ano == 2000), 'sequencial'].fillna(df.loc[(df.ano == 1996) | (df.ano == 2000), 'numero'])

In [496]:
df_turno = pd.merge( df, vote[['ano', 'id_municipio', 'sequencial_candidato', 'turno']], 
                how = 'left', right_on = ['ano', 'id_municipio', 'sequencial_candidato'], left_on = ['ano', 'id_municipio', 'sequencial'])

In [515]:
vote['annie_id'] = vote['ano'].astype(str) + vote['id_municipio'] + vote['sequencial_candidato'] + vote['turno'].astype(str)

df_turno['annie_id'] = df_turno['ano'].astype(str) + df_turno['id_municipio'] + df_turno['sequencial'] + df_turno['turno'].astype(str)
df_turno['annie_id'] = df_turno['annie_id'].apply(lambda x: x.split('.')[0])
#df['annie_id'] = df['ano'].astype(str) + df['id_municipio'] + df['numero'] 

In [428]:
vote.shape

(96841, 14)

In [429]:
vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,41,NaN,nao eleito,2388,19962704302411
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,16,NaN,nao eleito,1713,19961302603161
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,26,NaN,nao eleito,15721,19962927408261
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,29,NaN,nao eleito,705,19962927408291
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,30,NaN,nao eleito,989,19962927408301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96836,2020,2,eleicao ordinaria,CE,2303709,13730,prefeito,PROS,90,60000718891,441082,eleito,83588,20202303709600007188912
96837,2020,2,eleicao ordinaria,CE,2304400,13897,prefeito,PROS,90,60000653316,439843,nao eleito,624892,20202304400600006533162
96838,2004,1,eleicao ordinaria,SE,2802601,31518,prefeito,PP,11,88,NaN,eleito,2065,20042802601881
96839,2004,1,eleicao ordinaria,SE,2803609,31712,prefeito,PTB,14,14,119425,nao eleito,5530,20042803609141


In [430]:
#fill in nas from vote

#print(vote[(vote.id_candidato_bd.isna()) & (vote.ano > 2000)].shape[0])
cand_dir = pd.merge(vote1, df, how = 'left', left_on = ['id_municipio', 'ano', 'sequencial_candidato'], right_on = ['id_municipio', 'ano', 'sequencial'])
cand_dir['id_candidato_bd_full'] = cand_dir['id_candidato_bd_x']
cand_dir['id_candidato_bd_full'] = cand_dir['id_candidato_bd_full'].fillna(cand_dir['id_candidato_bd_y'])

cand_dir = cand_dir[['ano', 'id_municipio', 'sequencial_candidato', 'id_candidato_bd_full']]
cand_dir = cand_dir.drop_duplicates()

#print(cand_dir[cand_dir.id_candidato_bd_full.isna() & (cand_dir.ano > 2000)].shape[0])

vote = pd.merge(vote, cand_dir, how = 'left', on = ['id_municipio', 'ano', 'sequencial_candidato'])
df = pd.merge(df, cand_dir, how = 'left', left_on = ['id_municipio', 'ano', 'sequencial'], right_on = ['id_municipio', 'ano', 'sequencial_candidato'])


In [431]:
vote.shape

(96841, 15)

In [432]:
vote.columns = ['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd_old', 'resultado', 'votos',
       'annie_id', 'id_candidato_bd_full']

vote.columns = ['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd_old', 'resultado', 'votos',
       'annie_id', 'id_candidato_bd']

vote = vote[['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado', 'votos',
       'annie_id', 'id_candidato_bd_old']]


In [433]:
vote.shape

(96841, 15)

In [434]:
#vote = vote.drop_duplicates()

#df = df.drop_duplicates()

# vote = vote.drop_duplicates(['ano', 'id_municipio', 'numero_candidato', 'turno', 'tipo_eleicao'])
# df = df.drop_duplicates(['ano', 'id_municipio', 'numero'])

vote = vote.drop_duplicates(['ano', 'id_municipio', 'sequencial_candidato', 'turno', 'tipo_eleicao'])
# df = df.drop_duplicates(['ano', 'id_municipio', 'numero'])

In [435]:
vote.shape

(96841, 15)

In [436]:
#create variables that we're calculating below.

final = vote[['ano', 'id_municipio']].drop_duplicates().reset_index(drop = True)

final['runoff'] = np.zeros(final.shape[0])

final['winner_turno1'] = np.full([final.shape[0]], np.nan)
final['winner_turno2'] = np.full([final.shape[0]], np.nan)

final['totalvotes_turno1'] = np.zeros(final.shape[0])
final['totalvotes_turno2'] = np.zeros(final.shape[0])

final['winner_votes_turno_1'] = np.zeros(final.shape[0])
final['winner_votes_turno_2'] = np.zeros(final.shape[0])

final['margin_turno1'] = np.zeros(final.shape[0])
final['margin_turno2'] = np.zeros(final.shape[0])

final['incumbant'] = np.zeros(final.shape[0])
final['term_limited_seat'] = np.zeros(final.shape[0])

final['overall_winner'] = np.zeros(final.shape[0])

final['tipo'] = np.full([final.shape[0]], 'eleicao ordinaria')
final['special'] = np.zeros(final.shape[0])

final['overall_winner_numero_candidato'] = np.zeros(final.shape[0])




In [437]:
#handle 1996 and 2000 NA values
#incumbant, term limited seat

#first, incumbant

#Who won in 1996? did they run in 2000?
#Who won in 2000? did they run in 2004? 

#next term limited
#who won in 1996 did they win in 2000 again? mark as term limited in 2004.





In [578]:
# url_1996_1 = 'https://gist.githubusercontent.com/aulichney/88154f3d2e7bbf078032c5bede660cb0/raw/baa0bec7b690075e11c0765dc74575234e7e342a/resultado_1996_turno1.csv'
# url_1996_2 = 'https://gist.githubusercontent.com/aulichney/ec7bc2c9d454c2ae9850601e6ff46b43/raw/ee81de423ba17c36aad721d390df8617c4aa1ec1/resultado_1996_turno2.csv'

# url_2000_1 = 'https://gist.githubusercontent.com/aulichney/b0f10ec9cc24417d653792e01ac81719/raw/9835dad8f7aa06e4428ee0c0a404fbab2d132772/resultado_2000_turno1.csv'
# url_2000_2 = 'https://gist.githubusercontent.com/aulichney/25c535ec3a32e228dbfac973be5d64a8/raw/c43e819ae0c7ec11ecbf5d19185698c3fbb9270a/resultado_2000_turno2.csv'

# url_2004_1 = 'test'
# url_2004_2 = 'test'

url_1996_1 = f'/Users/annieulichney/Documents/GitHub/Deforestation/ElectionData/1996_turno1.csv'
url_1996_2 = f'/Users/annieulichney/Documents/GitHub/Deforestation/ElectionData/1996_turno2.csv'

url_2000_1 = f'/Users/annieulichney/Documents/GitHub/Deforestation/ElectionData/2000_turno1.csv'
url_2000_2 = f'/Users/annieulichney/Documents/GitHub/Deforestation/ElectionData/2000_turno2.csv'

url_2004_1 = f'/Users/annieulichney/Documents/GitHub/Deforestation/ElectionData/2004_turno1.csv'
url_2004_2 = f'/Users/annieulichney/Documents/GitHub/Deforestation/ElectionData/2004_turno2.csv'

df_1996_1 = pd.read_csv(url_1996_1).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_1996_2 = pd.read_csv(url_1996_2).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_1996 = pd.concat([df_1996_1, df_1996_2])

df_2000_1 = pd.read_csv(url_2000_1).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_2000_2 = pd.read_csv(url_2000_2).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_2000 = pd.concat([df_2000_1, df_2000_2])

df_2004_1 = pd.read_csv(url_2004_1).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_2004_2 = pd.read_csv(url_2004_2).drop(['Unnamed: 10', 'Coligação', 'Cargo'], axis = 1)
df_2004 = pd.concat([df_2004_1, df_2004_2])



In [440]:
# url_1996_1 = 'https://gist.githubusercontent.com/aulichney/88154f3d2e7bbf078032c5bede660cb0/raw/baa0bec7b690075e11c0765dc74575234e7e342a/resultado_1996_turno1.csv'
# url_1996_2 = 'https://gist.githubusercontent.com/aulichney/ec7bc2c9d454c2ae9850601e6ff46b43/raw/ee81de423ba17c36aad721d390df8617c4aa1ec1/resultado_1996_turno2.csv'

# url_2000_1 = 'https://gist.githubusercontent.com/aulichney/b0f10ec9cc24417d653792e01ac81719/raw/9835dad8f7aa06e4428ee0c0a404fbab2d132772/resultado_2000_turno1.csv'
# url_2000_2 = 'https://gist.githubusercontent.com/aulichney/25c535ec3a32e228dbfac973be5d64a8/raw/c43e819ae0c7ec11ecbf5d19185698c3fbb9270a/resultado_2000_turno2.csv'

# year = 1996
# raw_turno1 = pd.read_csv(url_1996_1, header = 1)
# raw_turno1['turno'] = np.repeat(1, raw_turno1.shape[0])

# raw_turno2 = pd.read_csv(url_1996_2, header = 1)
# raw_turno2['turno'] = np.repeat(2, raw_turno2.shape[0])

# raw = pd.concat([raw_turno1, raw_turno2])
# raw.columns = ['cargo', 'sigla_uf', 'muni_name', 'sigla_partido', 'numero_candidato', 'cand_name',
#        'votos', 'resultado', 'pct', 'coligacao', 'drop1',
#        'turno']

# raw = raw[['cargo', 'sigla_uf', 'muni_name', 'sigla_partido', 'numero_candidato', 'cand_name',
#        'votos', 'resultado', 'pct', 'coligacao', 'turno']]

# raw['ano'] = year

# raw['muni_name'] = raw['muni_name'].apply(unidecode).apply(lambda x: x.lower())
# raw['cand_name'] = raw['cand_name'].apply(unidecode).apply(lambda x: x.lower())
# raw['resultado'] = raw['resultado'].apply(unidecode).apply(lambda x: x.lower())
# raw['cargo'] = raw['cargo'].apply(unidecode).apply(lambda x: x.lower())
# raw['votos'] = raw['votos'].apply(lambda x: x.replace(',', ''))
# raw['muni_name'] = raw['muni_name'].apply(lambda x: x.replace('\'', ''))
# raw['muni_name'] = raw['muni_name'].apply(lambda x: x.replace('-', ''))
# raw['muni_name'] = raw['muni_name'].apply(lambda x: x.replace(' ', ''))

# for col in raw.columns:
#     if type(raw[col][0]) == str:
#         raw[col] = raw[col].apply(unidecode).apply(lambda x: x.lower())

# raw['sigla_uf'] = raw['sigla_uf'].apply(unidecode).apply(lambda x: x.lower())

In [441]:
url_dir = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/diretorio.Rdata'
dir = pyreadr.read_r(url_dir)['df.diretorio']
dir = dir[['municipio', 'sigla_uf', 'id_municipio', 'id_municipio_6', 'id_municipio_tse', 'id_municipio_rf', 'id_municipio_bcb']]
dir.municipio = dir.municipio.apply(lambda x: x.replace('\'', ''))
dir.municipio = dir.municipio.apply(lambda x: x.replace('-', ''))
dir.municipio = dir.municipio.apply(lambda x: x.replace(' ', ''))

for col in dir.columns:
    if type(dir[col][0]) == str:
        dir[col] = dir[col].apply(unidecode).apply(lambda x: x.lower())
    
#raw_merged = pd.merge(raw, dir, how = 'left', left_on = ['sigla_uf', 'muni_name'], right_on = ['sigla_uf', 'municipio'])
#raw_merged[raw_merged.id_municipio_tse.isna()].shape

(396, 18)

In [442]:
convert_muni = pd.read_csv('https://raw.githubusercontent.com/ekatovich/Municipality_Name_ID_crosswalk/master/Raw_Brazil_GeographicalUnits.csv', encoding='latin-1')
convert_muni = convert_muni[['uf', 'uf_name', 'mesoregion_stub', 'mesoregion_name',
       'microregion_stub', 'microregion_name', 'munic_stub', 'munic_code',
       'munic_name']]
convert_muni.munic_code = convert_muni.munic_code.astype(str)

for col in convert_muni.columns:
    if type(convert_muni[col][0]) == str:
        convert_muni[col] = convert_muni[col].apply(unidecode).apply(lambda x: x.lower())

convert_muni.munic_name = convert_muni.munic_name.apply(lambda x: x.replace('\'', ''))
convert_muni.munic_name = convert_muni.munic_name.apply(lambda x: x.replace('-', ''))
convert_muni.munic_name = convert_muni.munic_name.apply(lambda x: x.replace(' ', ''))
        
for col in convert_muni.columns:
    if type(convert_muni[col][0]) == str:
        convert_muni[col] = convert_muni[col].apply(unidecode).apply(lambda x: x.lower())

In [579]:
convert_muni = pd.read_csv('https://raw.githubusercontent.com/ekatovich/Municipality_Name_ID_crosswalk/master/Raw_Brazil_GeographicalUnits.csv', encoding='latin-1')
convert_muni = convert_muni[['uf', 'uf_name', 'mesoregion_stub', 'mesoregion_name',
       'microregion_stub', 'microregion_name', 'munic_stub', 'munic_code',
       'munic_name']]
convert_muni.munic_code = convert_muni.munic_code.astype(str)

df_only_1996 = df[(df.ano == 1996)]
vote_only_1996 = vote[(vote.resultado == 'eleito') & (vote.ano == 1996)]
winners_1996 = pd.merge(vote_only_1996, df_only_1996, how = 'left', left_on = ['ano', 'id_municipio', 'numero_candidato'], right_on = ['ano', 'id_municipio', 'numero'])

df_only_2000 = df[(df.ano == 2000)]
vote_only_2000 = vote[(vote.resultado == 'eleito') & (vote.ano == 2000)]
winners_2000 = pd.merge(vote_only_2000, df_only_2000, how = 'left', left_on = ['ano', 'id_municipio', 'numero_candidato'], right_on = ['ano', 'id_municipio', 'numero'])

df_only_2004 = df[(df.ano == 2004)]
vote_only_2004 = vote[(vote.resultado == 'eleito') & (vote.ano == 2004)]
winners_2004 = pd.merge(vote_only_2004, df_only_2004, how = 'left', left_on = ['ano', 'id_municipio', 'numero_candidato'], right_on = ['ano', 'id_municipio', 'numero'])

#add muni name to winners data
winners_1996 = pd.merge(winners_1996, convert_muni[['munic_code', 'munic_name']], how = 'left', left_on = 'id_municipio', right_on = 'munic_code')
winners_2000 = pd.merge(winners_2000, convert_muni[['munic_code', 'munic_name']], how = 'left', left_on = 'id_municipio', right_on = 'munic_code')
winners_2004 = pd.merge(winners_2004, convert_muni[['munic_code', 'munic_name']], how = 'left', left_on = 'id_municipio', right_on = 'munic_code')

#get just the winner names
df_1996 = df_1996[df_1996['Situação'] == 'Eleito']
df_1996 = df_1996[['Município', 'Candidato','Votos Nominais', 'Situação']]
df_1996['Município'] = df_1996['Município'].apply(unidecode).apply(lambda x: x.lower())
winners_1996.munic_name = winners_1996.munic_name.apply(unidecode).apply(lambda x: x.lower())
df_1996['Candidato'] = df_1996['Candidato'].apply(unidecode).apply(lambda x: x.lower())

df_2000 = df_2000[df_2000['Situação'] == 'Eleito']
df_2000 = df_2000[['Município', 'Candidato','Votos Nominais', 'Situação']]
df_2000['Município'] = df_2000['Município'].apply(unidecode).apply(lambda x: x.lower())
winners_2000.munic_name = winners_2000.munic_name.apply(unidecode).apply(lambda x: x.lower())
df_2000['Candidato'] = df_2000['Candidato'].apply(unidecode).apply(lambda x: x.lower())

df_2004_all = df_2004.copy()

df_2004 = df_2004[df_2004['Situação'] == 'Eleito']
df_2004 = df_2004[['Município', 'Candidato','Votos Nominais', 'Situação']]
df_2004['Município'] = df_2004['Município'].apply(unidecode).apply(lambda x: x.lower())
winners_2004.munic_name = winners_2004.munic_name.apply(unidecode).apply(lambda x: x.lower())
df_2004['Candidato'] = df_2004['Candidato'].apply(unidecode).apply(lambda x: x.lower())

df_2004_all = df_2004_all[['Município', 'Candidato','Votos Nominais', 'Situação']]
df_2004_all['Município'] = df_2004_all['Município'].apply(unidecode).apply(lambda x: x.lower())
df_2004_all['Candidato'] = df_2004_all['Candidato'].apply(unidecode).apply(lambda x: x.lower())

#get the names of the winners of all elections in 1996, 2000, 2004
winners_1996_names = pd.merge(winners_1996, df_1996, how = 'left', left_on = 'munic_name', right_on = 'Município')
winners_1996_names['won_1996'] = np.ones(winners_1996_names.shape[0])
winners_1996_subset = winners_1996_names[['id_municipio', 'munic_name', 'Candidato', 'won_1996']]

winners_2000_names = pd.merge(winners_2000, df_2000, how = 'left', left_on = 'munic_name', right_on = 'Município')
winners_2000_names['won_2000'] = np.ones(winners_2000_names.shape[0])
winners_2000_subset = winners_2000_names[['id_municipio', 'munic_name', 'Candidato', 'won_2000']]

winners_2004_names = pd.merge(winners_2004, df_2004, how = 'left', left_on = 'munic_name', right_on = 'Município')
winners_2004_names['won_2004'] = np.ones(winners_2004_names.shape[0])
winners_2004_subset = winners_2004_names[['id_municipio', 'munic_name', 'Candidato', 'won_2004']]


#1996 winners that also won in 2000
term_limited_2004 = pd.merge( winners_1996_subset, df_2000, how = 'inner', left_on = ['Candidato'], right_on = ['Candidato']).id_municipio

#2000 winners that also won in 2004
term_limited_2008 = pd.merge( winners_2000_subset, df_2004, how = 'inner', left_on = ['Candidato'], right_on = ['Candidato']).id_municipio

#incumbant in 2004 is someone that won in 2000 and ran in 2004
incumbant_2004 = pd.merge(winners_2000_subset, df_2004_all, how = 'inner', left_on = ['munic_name', 'Candidato'], right_on = ['Município', 'Candidato']).id_municipio

for muni in term_limited_2004:
    final.loc[final[(final.ano == 2004) & (final.id_municipio == muni)].index, 'term_limited_seat'] = 1

for muni in term_limited_2008:
    final.loc[final[(final.ano == 2008) & (final.id_municipio == muni)].index, 'term_limited_seat'] = 1

for muni in incumbant_2004:
    final.loc[final[(final.ano == 2004) & (final.id_municipio == muni)].index, 'incumbant'] = 1





In [446]:
final = final.sort_values( by = ['ano'], axis = 0)

In [447]:
final = final.reset_index(drop = True)

In [448]:
vote.shape

(96841, 15)

In [449]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,41,NaN,nao eleito,2388,19962704302411,NaN
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,16,NaN,nao eleito,1713,19961302603161,NaN
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,26,NaN,nao eleito,15721,19962927408261,NaN
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,29,NaN,nao eleito,705,19962927408291,NaN
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,30,NaN,nao eleito,989,19962927408301,NaN


In [450]:
#write over the type for this muni year which is the only 2 term special election type!
vote.loc[vote[(vote.id_municipio == '1600303')&(vote.ano == 2020)].index, 'tipo_eleicao'] = 'eleicao ordinaria'

#for i in tqdm(range(final.shape[0])): 
error_messages = []
missing_ano = []
missing_muni = []

for i in tqdm(range(final.shape[0])):   
    this_ano = final.iloc[i]['ano']
    this_muni = final.iloc[i]['id_municipio']

    did_runoff_occur = 0
    overall_winner = "None"

    if this_ano == 1996:
        continue
    if this_ano == 2000:
        continue

    this_vote = vote[(vote.ano == this_ano) & (vote.id_municipio == this_muni)]


    ##GET TURNO 1 WINNER AND MARGIN AND TOTAL VOTES

    #winner for ordinary election turno 1
    winner_ordinary_turno1 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

    if winner_ordinary_turno1.shape[0] > 1: #more than one possible winner for turno 1
        if '2º turno' in list(winner_ordinary_turno1.resultado): #find the max votes in turno 1
            final.loc[i, 'winner_turno1'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
            final.loc[i, 'winner_turno1_sequencial_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'sequencial_candidato']
            final.loc[i, 'winner_turno1_numero_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'numero_candidato']

            #margin for exactly one winner for turno 1.
            number_of_votes = list(this_vote[this_vote.turno == 1].sort_values(by = 'votos', axis = 0, ascending = False).votos)
            if len(number_of_votes) == 1:
                final.loc[i, 'margin_turno1'] = number_of_votes[0] #margin
            else:
                final.loc[i, 'margin_turno1'] = number_of_votes[0] - number_of_votes[1] #margin
            
            final.loc[i, 'winner_votes_turno_1'] = number_of_votes[0] #winner number of votes
            final.loc[i, 'totalvotes_turno1'] = sum(number_of_votes) #total number of votes in turno 1

            did_runoff_occur = 1
        else: 
            error_messages.append("More than one turno 1 winner ordinary election for " + str(this_muni) + " " + str(this_ano))
    elif winner_ordinary_turno1.shape[0] == 0:  #missing winner for ordinary election! 
        winner_other_type = vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)&(vote.resultado.isin(['eleito', 'eleito por qp']))]
        if list(winner_other_type.tipo_eleicao):
            final.loc[i, 'special'] = 1
            final.loc[i, 'winner_special'] = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
            final.loc[i, 'winner_special_sequencial_candidato'] = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'sequencial_candidato']
            final.loc[i, 'winner_special_numero_candidato'] = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'numero_candidato']

            outcome_type = np.unique(vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)].resultado)
            overall_winner = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
            overall_winner_annie_id = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'annie_id']
            overall_winner_sequencial_candidato = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'sequencial_candidato'] #overall_winner_sequencial_candidato
            overall_winner_numero_candidato = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'numero_candidato']

            final.loc[i, 'overall_winner'] = overall_winner
            final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
            final.loc[i, 'overall_winner_sequencial_candidato'] = overall_winner_sequencial_candidato
            final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato
            
            
            final.loc[i, 'tipo'] = list(winner_other_type.tipo_eleicao)[0]
            final.loc[i, 'outcomes'] = ", ".join(outcome_type)

            number_of_votes = list(vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)].sort_values(by = 'votos', axis = 0, ascending = False).votos)
            if winner_other_type.shape[0] == 1:
                if len(number_of_votes) == 1:
                    final.loc[i, 'margin_special'] = number_of_votes[0] #margin
                else:
                    final.loc[i, 'margin_special'] = number_of_votes[0] - number_of_votes[1] #margin
                
                #final.loc[i, 'winner_votes_turno_1'] = number_of_votes[0] #winner number of votes
                final.loc[i, 'totalvotes_special'] = sum(number_of_votes) #total number of votes in turno 1
            else:
                error_messages.append( str(winner_other_type.shape[0])  + 'special election winners for '+ str(this_muni) + " " + str(this_ano ) )
            
        else:
            error_messages.append("No winner turno 1 ordinary election for " + str(this_muni) + " " + str(this_ano ))
            missing_muni.append(this_muni)
            missing_ano.append(this_ano)

        this_vote[(this_vote.tipo_eleicao != 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

    else: #this is when there is exactly one winner
        final.loc[i, 'winner_turno1'] = list(winner_ordinary_turno1['id_candidato_bd'])[0]
        final.loc[i, 'winner_turno1_sequencial_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'sequencial_candidato']
        final.loc[i, 'winner_turno1_numero_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'numero_candidato']

        overall_winner = list(winner_ordinary_turno1.id_candidato_bd)[0]
        overall_winner_annie_id = list(winner_ordinary_turno1.annie_id)[0]
        overall_winner_sequencial_candidato = list(winner_ordinary_turno1.sequencial_candidato)[0]
        overall_winner_numero_candidato = list(winner_ordinary_turno1.numero_candidato)[0]

        final.loc[i, 'overall_winner'] = overall_winner
        final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
        final.loc[i, 'overall_winner_sequencial_candidato'] = overall_winner_sequencial_candidato
        final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato

        #margin for exactly one winner for turno 1.
        number_of_votes = list(this_vote[this_vote.turno == 1].sort_values(by = 'votos', axis = 0, ascending = False).votos)
        if len(number_of_votes) == 1:
            final.loc[i, 'margin_turno1'] = number_of_votes[0] #margin
        else:
            final.loc[i, 'margin_turno1'] = number_of_votes[0] - number_of_votes[1] #margin
        final.loc[i, 'winner_votes_turno_1'] = number_of_votes[0] #winner number of votes
        final.loc[i, 'totalvotes_turno1'] = sum(number_of_votes) #total number of votes in turno 1
    

        #if '2º turno' in list(winner_ordinary_turno1.resultado):
        #    print("Not sure if this is supposed to happen:", i, this_ano, this_muni) 
        #   did_runoff_occur = 1
    

    ## GET TURNO 2 WINNER AND MARGIN AND TOTAL VOTES

    if did_runoff_occur == 1: 
        
        winner_ordinary_turno2 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', 'eleito por qp'])) & (this_vote.turno == 2)]

        if winner_ordinary_turno2.shape[0] == 1:
            
            final.loc[i, 'winner_turno2'] = list(winner_ordinary_turno2.id_candidato_bd)[0]
            final.loc[i, 'winner_turno2_sequencial_candidato'] = list(winner_ordinary_turno2.sequencial_candidato)[0]
            final.loc[i, 'winner_turno2_numero_candidato'] = list(winner_ordinary_turno2.numero_candidato)[0]

            #use this for incumbant calculation
            overall_winner = list(winner_ordinary_turno2.id_candidato_bd)[0]
            overall_winner_annie_id = list(winner_ordinary_turno2.annie_id)[0]
            overall_winner_sequencial_candidato = list(winner_ordinary_turno1.sequencial_candidato)[0]
            overall_winner_numero_candidato = list(winner_ordinary_turno1.numero_candidato)[0]

            final.loc[i, 'overall_winner'] = overall_winner
            final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
            final.loc[i, 'overall_winner_sequencial_candidato'] = overall_winner_sequencial_candidato
            final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato

            #margin for exactly one winner for turno 1.
            number_of_votes_2 = list(this_vote[this_vote.turno == 2].sort_values(by = 'votos', axis = 0, ascending = False).votos)
            if len(number_of_votes_2) == 1:
                final.loc[i, 'margin_turno2'] = number_of_votes_2[0] #margin
            else:
                final.loc[i, 'margin_turno2'] = number_of_votes_2[0] - number_of_votes_2[1] #margin
            final.loc[i, 'winner_votes_turno_2'] = number_of_votes_2[0] #winner number of votes
            final.loc[i, 'totalvotes_turno2'] = sum(number_of_votes_2) #total number of votes in turno 1
        elif winner_ordinary_turno2.shape[0] > 1: 
            error_messages.append('More than one turno 2 winner orginary election for' + str(this_muni) + " " + str(this_ano))
        elif winner_ordinary_turno2.shape[0] == 0 : 
            error_messages.append('No turno 2 winner orginary election for' + str(this_muni) + " " + str(this_ano))


    #INCUMBANT
    if this_ano >= 2004:
        this_vote_4yr = vote[(vote.ano == this_ano-4)&(vote.id_municipio == this_muni)]

        if overall_winner in list(this_vote_4yr.id_candidato_bd):
            final.loc[i, 'incumbant'] = 1
    
    

    
with open(r'errors.txt', 'w') as fp:
    for item in error_messages:
        fp.write("%s\n" % item)
    print('Done')





#rewrite the 2 term special election
final.loc[(final.ano == 2020)&(final.id_municipio == '1600303'), 'tipo'] = 'eleicoes municipais 2020 - ap'



    

100%|██████████| 33479/33479 [10:46<00:00, 51.76it/s] 

Done


In [451]:
#final[final.tipo != 'eleicao ordinaria']

final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,overall_winner_annie_id,overall_winner_sequencial_candidato,winner_turno2_sequencial_candidato,winner_turno2_numero_candidato,winner_special,winner_special_sequencial_candidato,winner_special_numero_candidato,outcomes,margin_special,totalvotes_special
0,1996,2704302,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,2201606,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,2202174,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,2202752,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,2203255,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [452]:
final.loc[final[final.tipo == 'eleicao ordinaria'].index, 'special'] = 0
final.loc[final[final.tipo != 'eleicao ordinaria'].index, 'special'] = 1

final.loc[final[final.runoff == 0].index, 'winner'] = final.loc[final[final.runoff == 0].index, 'winner_turno1']
final.loc[final[final.runoff == 1].index, 'winner'] = final.loc[final[final.runoff == 1].index, 'winner_turno2']
final.loc[final[final.special == 1].index, 'winner'] = final.loc[final[final.special == 1].index, 'winner_special']

final.loc[final[final.runoff == 0].index, 'margin'] = final.loc[final[final.runoff == 0].index, 'margin_turno1']
final.loc[final[final.runoff == 1].index, 'margin'] = final.loc[final[final.runoff == 1].index, 'margin_turno2']
final.loc[final[final.special == 1].index, 'margin'] = final.loc[final[final.special == 1].index, 'margin_special']

final.loc[final[final.runoff == 0].index, 'totalvotes'] = final.loc[final[final.runoff == 0].index, 'totalvotes_turno1']
final.loc[final[final.runoff == 1].index, 'totalvotes'] = final.loc[final[final.runoff == 1].index, 'totalvotes_turno2']
final.loc[final[final.special == 1].index, 'totalvotes'] = final.loc[final[final.special == 1].index, 'totalvotes_special']

final.loc[final[final.runoff == 0].index, 'sequencial_candidato'] = final.loc[final[final.runoff == 0].index, 'winner_turno1_sequencial_candidato']
final.loc[final[final.runoff == 1].index, 'sequencial_candidato'] = final.loc[final[final.runoff == 1].index, 'winner_turno2_sequencial_candidato']
final.loc[final[final.special == 1].index, 'sequencial_candidato'] = final.loc[final[final.special == 1].index, 'winner_special_sequencial_candidato']


final.loc[final[final.runoff == 0].index, 'numero_candidato'] = final.loc[final[final.runoff == 0].index, 'winner_turno1_numero_candidato']
final.loc[final[final.runoff == 1].index, 'numero_candidato'] = final.loc[final[final.runoff == 1].index, 'winner_turno2_numero_candidato']
final.loc[final[final.special == 1].index, 'numero_candidato'] = final.loc[final[final.special == 1].index, 'winner_special_numero_candidato']




In [453]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,41,NaN,nao eleito,2388,19962704302411,NaN
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,16,NaN,nao eleito,1713,19961302603161,NaN
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,26,NaN,nao eleito,15721,19962927408261,NaN
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,29,NaN,nao eleito,705,19962927408291,NaN
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,30,NaN,nao eleito,989,19962927408301,NaN


In [454]:
remains_na = final[(final.ano > 2000 ) & (final.overall_winner_annie_id.isna()) & (final.ano  <= 2016 )]

remains_na = remains_na.reset_index()

error_messages2 = []
for i in range(remains_na.shape[0]):

    r_row = remains_na.iloc[i]


    r_vote = vote1[(vote1.ano == r_row.ano) & (vote1.id_municipio == r_row.id_municipio)]

    error_messages2.append('Only result for '+ r_row.id_municipio + ' in ' + str(r_row.ano) + ' is '  +  ", ".join(np.unique(r_vote.resultado)))

    with open(r'errors_final.txt', 'w') as fp:
        for item in error_messages2:
            fp.write("%s\n" % item)
        

In [468]:
final = final.drop(['overall_winner_4yrs', 'overall_winner_8yrs'], axis = 1)

KeyError: "['overall_winner_4yrs' 'overall_winner_8yrs'] not found in axis"

In [346]:
#double check on these ones to user sequencial candidato! 
final[((final.sequencial_candidato.isna()) & (final.ano > 2000) & (final.ano < 2020))]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,winner_turno2_sequencial_candidato,winner_special,winner_special_sequencial_candidato,outcomes,margin_special,totalvotes_special,winner,margin,totalvotes,sequencial_candidato
6926,2004,3301009,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,963200,323,"eleito, nao eleito",22054.0,240672.0,NaN,0.0,0.0,NaN
17830,2012,5203500,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
24475,2016,2803609,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
26088,2016,3545506,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


In [469]:
final_4yrs = final.copy()
final_4yrs.ano = final_4yrs.ano + 4
final_4yrs = final_4yrs[['ano', 'id_municipio', 'overall_winner']]
final_4yrs.columns = ['ano', 'id_municipio', 'overall_winner_4yrs']

final_8yrs = final.copy()
final_8yrs.ano = final_8yrs.ano + 8
final_8yrs = final_8yrs[['ano', 'id_municipio', 'overall_winner']]
final_8yrs.columns = ['ano', 'id_municipio', 'overall_winner_8yrs']

final = pd.merge(final, final_4yrs, how = 'left', on = ['ano', 'id_municipio'])
final = pd.merge(final, final_8yrs, how = 'left', on = ['ano', 'id_municipio'])

final['term_limited_seat'] = 0


In [470]:
final[(~final.overall_winner_4yrs.isna()) & (final.overall_winner_4yrs == final.overall_winner_8yrs) & (final.overall_winner_4yrs != 0)]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,margin,totalvotes,sequencial_candidato,numero_candidato,overall_winner_4yrs_x,overall_winner_8yrs_x,overall_winner_4yrs_y,overall_winner_8yrs_y,overall_winner_4yrs,overall_winner_8yrs
16789,2012,1718659,0.0,660519,NaN,1462.0,0.0,786.0,0.0,110.0,...,110.0,1462.0,270000005300,15,1383914,1383914,1383914,1383914,1383914,1383914
16790,2012,1718709,0.0,1627991,NaN,1786.0,0.0,988.0,0.0,190.0,...,190.0,1786.0,270000005925,15,1784479,1784479,1784479,1784479,1784479,1784479
16791,2012,1718865,0.0,1283689,NaN,4141.0,0.0,1907.0,0.0,385.0,...,385.0,4141.0,270000008397,40,1611183,1611183,1611183,1611183,1611183,1611183
16794,2012,1718840,0.0,502812,NaN,2940.0,0.0,1322.0,0.0,282.0,...,282.0,2940.0,270000005605,22,1083043,1083043,1083043,1083043,1083043,1083043
16799,2012,1716307,0.0,733079,NaN,2714.0,0.0,1593.0,0.0,472.0,...,472.0,2714.0,270000002992,15,630398,630398,630398,630398,630398,630398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33462,2020,2514602,0.0,1022013,NaN,3109.0,0.0,2076.0,0.0,1273.0,...,1273.0,3109.0,150000741774,10,1596245,1596245,1596245,1596245,1596245,1596245
33466,2020,2603207,0.0,605460,NaN,13499.0,0.0,9634.0,0.0,5769.0,...,5769.0,13499.0,170000940868,10,1437280,1437280,1437280,1437280,1437280,1437280
33474,2020,2516755,0.0,1220630,NaN,2306.0,0.0,1616.0,0.0,926.0,...,926.0,2306.0,150000654766,10,1796509,1796509,1796509,1796509,1796509,1796509
33475,2020,2516706,0.0,173648,NaN,9338.0,0.0,4677.0,0.0,727.0,...,727.0,9338.0,150000638554,12,499416,499416,499416,499416,499416,499416


In [471]:
final.loc[final[(~final.overall_winner_4yrs.isna()) & (final.overall_winner_4yrs == final.overall_winner_8yrs) & (final.overall_winner_4yrs != 0)].index, 'term_limited_seat'] = 1

In [416]:
vote1[(vote1.ano.isin([2008, 2012, 2004, 2000, 1996])) & (vote1.id_municipio == '2201176') & (vote1.resultado == 'eleito')]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id
35363,2008,1,eleicao ordinaria,PI,2201176,10600,prefeito,PMDB,15,6786,47552,eleito,1112,2008220117667861
71240,2004,1,eleicao ordinaria,PI,2201176,10600,prefeito,PSB,40,109,630544,eleito,1177,200422011761091
73247,2012,1,eleicao ordinaria,PI,2201176,10600,prefeito,PSB,40,180000004349,669478,eleito,1326,201222011761800000043491
76967,1996,1,eleicao ordinaria,PI,2201176,10600,prefeito,PSDB,45,45,NaN,eleito,1093,19962201176451
78187,2000,1,eleicao ordinaria,PI,2201176,10600,prefeito,PSDB,45,45,1458321,eleito,1594,20002201176451


In [472]:
sum(final.term_limited_seat == 1)
#used to be 4691

4691

In [473]:
final[(final.term_limited_seat == 1)]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,margin,totalvotes,sequencial_candidato,numero_candidato,overall_winner_4yrs_x,overall_winner_8yrs_x,overall_winner_4yrs_y,overall_winner_8yrs_y,overall_winner_4yrs,overall_winner_8yrs
16789,2012,1718659,0.0,660519,NaN,1462.0,0.0,786.0,0.0,110.0,...,110.0,1462.0,270000005300,15,1383914,1383914,1383914,1383914,1383914,1383914
16790,2012,1718709,0.0,1627991,NaN,1786.0,0.0,988.0,0.0,190.0,...,190.0,1786.0,270000005925,15,1784479,1784479,1784479,1784479,1784479,1784479
16791,2012,1718865,0.0,1283689,NaN,4141.0,0.0,1907.0,0.0,385.0,...,385.0,4141.0,270000008397,40,1611183,1611183,1611183,1611183,1611183,1611183
16794,2012,1718840,0.0,502812,NaN,2940.0,0.0,1322.0,0.0,282.0,...,282.0,2940.0,270000005605,22,1083043,1083043,1083043,1083043,1083043,1083043
16799,2012,1716307,0.0,733079,NaN,2714.0,0.0,1593.0,0.0,472.0,...,472.0,2714.0,270000002992,15,630398,630398,630398,630398,630398,630398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33462,2020,2514602,0.0,1022013,NaN,3109.0,0.0,2076.0,0.0,1273.0,...,1273.0,3109.0,150000741774,10,1596245,1596245,1596245,1596245,1596245,1596245
33466,2020,2603207,0.0,605460,NaN,13499.0,0.0,9634.0,0.0,5769.0,...,5769.0,13499.0,170000940868,10,1437280,1437280,1437280,1437280,1437280,1437280
33474,2020,2516755,0.0,1220630,NaN,2306.0,0.0,1616.0,0.0,926.0,...,926.0,2306.0,150000654766,10,1796509,1796509,1796509,1796509,1796509,1796509
33475,2020,2516706,0.0,173648,NaN,9338.0,0.0,4677.0,0.0,727.0,...,727.0,9338.0,150000638554,12,499416,499416,499416,499416,499416,499416


In [476]:
final_cut = final[(final.ano > 2000) & (final.ano < 2020)]

In [479]:
final_cut.shape

(22252, 40)

In [483]:
final_cut[final_cut.overall_winner_annie_id.isna()]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,margin,totalvotes,sequencial_candidato,numero_candidato,overall_winner_4yrs_x,overall_winner_8yrs_x,overall_winner_4yrs_y,overall_winner_8yrs_y,overall_winner_4yrs,overall_winner_8yrs
17830,2012,5203500,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,1024004,123954,1024004,123954,1024004,123954
24475,2016,2803609,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,502673,119425,502673,119425,502673,119425
26088,2016,3545506,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,1313566,1313566,1313566,1313566,1313566,1313566


In [517]:
demo_df = df_turno[['ano', 'sequencial', 'numero', 'numero_partido', 'sigla_partido', 'ocupacao', 'municipio_nascimento', 'genero', 'idade', 'annie_id']]

In [518]:
merged = pd.merge(final_cut, demo_df, how = 'left', left_on = 'overall_winner_annie_id', right_on = 'annie_id')

In [519]:
demo_df[demo_df.annie_id == '20043203106391']

,ano,sequencial,numero,numero_partido,sigla_partido,ocupacao,municipio_nascimento,genero,idade,annie_id
58620,2004,39,23,23,PPS,pecuarista,Jeronimo Monteiro,masculino,67,20043203106391


In [491]:
final_cut.overall_winner_annie_id

5671               20042102200471
5672               20043203007311
5673               20043203106391
5674              200432031631271
5675              200452148382391
                   ...           
27918    201633053071900000146221
27919    201621102031000000132951
27920    201621129021000000018921
27921    201621109061000000043131
27922    201621102781000000108981
Name: overall_winner_annie_id, Length: 22252, dtype: object

In [520]:
merged

,ano_x,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,ano_y,sequencial,numero,numero_partido,sigla_partido,ocupacao,municipio_nascimento,genero,idade,annie_id
0,2004,2102200,0.0,1562413,NaN,11941.0,0.0,6582.0,0.0,1775.0,...,2004.0,47,25,25,PFL,agricultor,Buriti,masculino,45,20042102200471
1,2004,3203007,0.0,1588551,NaN,15620.0,0.0,10613.0,0.0,5606.0,...,2004.0,31,23,23,PPS,engenheiro,Belo Horizonte,masculino,50,20043203007311
2,2004,3203106,0.0,1302803,NaN,6986.0,0.0,4176.0,0.0,1366.0,...,2004.0,39,23,23,PPS,pecuarista,Jeronimo Monteiro,masculino,67,20043203106391
3,2004,3203163,0.0,258030,NaN,7412.0,0.0,5146.0,0.0,2880.0,...,2004.0,127,25,25,PFL,industrial,Afonso Cláudio,masculino,47,200432031631271
4,2004,5214838,0.0,1773110,NaN,6027.0,0.0,2638.0,0.0,684.0,...,2004.0,239,45,45,PSDB,presidente da republica ministro de estado gov...,Tiros,masculino,40,200452148382391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22247,2016,3305307,0.0,858234,NaN,6281.0,0.0,4170.0,0.0,2059.0,...,2016.0,190000014622,11,11,PP,servidor publico municipal,Sao Sebastiao Do Alto,masculino,44,201633053071900000146221
22248,2016,2110203,0.0,146555,NaN,18042.0,0.0,15010.0,0.0,12876.0,...,2016.0,100000013295,65,65,PC do B,medico,Pastos Bons,masculino,51,201621102031000000132951
22249,2016,2112902,0.0,1510338,NaN,19317.0,0.0,5876.0,0.0,111.0,...,2016.0,100000001892,15,15,MDB,bibliotecario,Vargem Grande,feminino,59,201621129021000000018921
22250,2016,2110906,0.0,749655,NaN,6981.0,0.0,4270.0,0.0,1559.0,...,2016.0,100000004313,65,65,PC do B,empresario,São Francisco Do Maranhão,masculino,30,201621109061000000043131


In [478]:
df_turno

,ano,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,id_candidato_bd,cpf,titulo_eleitoral,sequencial,numero,...,instrucao,estado_civil,nacionalidade,sigla_uf_nascimento,municipio_nascimento,email,raca,sequencial_candidato,turno,annie_id
0,1996,eleicao ordinaria,PI,2211001,12190,NaN,NaN,NaN,16,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,1.0,19962211001161.0
1,1996,eleicao ordinaria,SE,2800308,31054,NaN,NaN,NaN,16,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,1.0,19962800308161.0
2,1996,eleicao ordinaria,RS,4314902,88013,NaN,NaN,NaN,56,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,1.0,19964314902561.0
3,1996,eleicao ordinaria,MG,3106200,41238,NaN,NaN,NaN,16,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,1.0,19963106200161.0
4,1996,eleicao ordinaria,PB,2507507,20516,NaN,NaN,NaN,16,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,1.0,19962507507161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100791,2020,eleicao ordinaria,MT,5106802,91359,1760144,30827310900,018444551856,110001042327,90,...,ensino medio completo,divorciado(a),brasileira,RS,Terra De Areia,plananorte@gmailcom,branca,110001042327,1.0,202051068021100010423271.0
100792,2020,eleicao ordinaria,PE,2613404,25674,1763982,31008089400,037956490833,170000760838,90,...,ensino superior completo,casado(a),brasileira,AL,São Luís Do Quitunde,aroldopajau@hotmailcom,branca,170000760838,1.0,202026134041700007608381.0
100793,2020,eleicao ordinaria,AM,1301209,2259,1768332,31221130234,012015422232,40000686733,90,...,ensino fundamental completo,solteiro(a),brasileira,PB,Juazeirinho,lucianodefarias01amldf@gmailcom,parda,NaN,NaN,2020130120940000686733nan
100794,2020,eleicao ordinaria,RJ,3306107,59218,1784783,32093632772,067680040361,190000995787,90,...,ensino superior completo,casado(a),brasileira,RJ,Valença,pimentadacunha@uolcombr,preta,190000995787,1.0,202033061071900009957871.0


In [248]:
missing = final[(final.ano >= 2004) & (final.ano < 2020) & (final.overall_winner.isna())]

In [250]:
missing

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,margin_special,totalvotes_special,winner,margin,totalvotes,sequencial_candidato,overall_winner_annie_id_4yrs,overall_winner_annie_id_8yrs,overall_winner_4yrs,overall_winner_8yrs
5730,2004,2306801,0.0,NaN,NaN,6170.0,0.0,3155.0,0.0,140.0,...,NaN,NaN,NaN,140.0,6170.0,38,NaN,NaN,0.0,NaN
5813,2004,2919504,0.0,NaN,NaN,22643.0,0.0,11663.0,0.0,683.0,...,NaN,NaN,NaN,683.0,22643.0,47,NaN,NaN,0.0,NaN
5886,2004,4110607,0.0,NaN,NaN,9771.0,0.0,5732.0,0.0,1693.0,...,NaN,NaN,NaN,1693.0,9771.0,63,NaN,NaN,0.0,NaN
5940,2004,3126406,0.0,NaN,NaN,1949.0,0.0,1014.0,0.0,443.0,...,NaN,NaN,NaN,443.0,1949.0,203,NaN,NaN,0.0,NaN
5970,2004,3138302,0.0,NaN,NaN,2318.0,0.0,1390.0,0.0,499.0,...,NaN,NaN,NaN,499.0,2318.0,1,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26819,2016,1508084,0.0,NaN,NaN,20152.0,0.0,10850.0,0.0,4400.0,...,NaN,NaN,NaN,4400.0,20152.0,140000005967,201215080841400000071181,2004150808491,NaN,1593176
26829,2016,2904852,0.0,NaN,NaN,9124.0,0.0,4566.0,0.0,8.0,...,NaN,NaN,NaN,8.0,9124.0,50000002361,20122904852500000278581,20042904852741,NaN,629000
27292,2016,2706000,0.0,NaN,NaN,7011.0,0.0,3511.0,0.0,11.0,...,NaN,NaN,NaN,11.0,7011.0,20000001657,20122706000200000055501,20042706000711,615450,629598
27833,2016,2409209,0.0,NaN,NaN,3125.0,0.0,2109.0,0.0,1148.0,...,NaN,NaN,NaN,1148.0,3125.0,200000002537,201224092092000000082031,20042409209961,157867,1799889


,ano,id_municipio,overall_winner_annie_id
0,2000,2704302,NaN
1,2000,2201606,NaN
2,2000,2202174,NaN
3,2000,2202752,NaN
4,2000,2203255,NaN
...,...,...,...
33474,2024,2516755,202025167551500006547661
33475,2024,2516706,202025167061500006385541
33476,2024,2516607,202025166071500011990431
33477,2024,2601805,202026018051700007647991


In [211]:
i = 5671
print(final.iloc[i].overall_winner_annie_id)

i = 5670
print(final.iloc[i].overall_winner_annie_id)

i = 5669
print(final.iloc[i].overall_winner_annie_id)


20042102200471
nan
nan


In [212]:
final.iloc[5671]['ano']

2004

In [213]:
# i = 5671

# this_winner = final.iloc[i]['overall_winner_annie_id']
# this_year = final.iloc[i]['ano']
# this_mun = final.iloc[i]['id_municipio']


# four_years_later = final[(final.ano == this_year + 4) & (final.id_municipio == this_mun)]
# eight_years_later = final[(final.ano == this_year + 8) & (final.id_municipio == this_mun)]

# winner_4yr = four_years_later.overall_winner_annie_id

# if len(this_winner) > 1:
#     print(i, ' failed!')


# if winner_4yr == this_winner:
#     final.loc[eight_years_later.index[0], 'term_limited'] = 1

5671  failed!


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [207]:
#i = 1957

for i in tqdm(range((final.shape[0]))):
    this_winner = final.iloc[i]['overall_winner_annie_id']
    this_year = final.iloc[i]['ano']
    this_mun = final.iloc[i]['id_municipio']

    if this_year > 2012:
        continue

    if this_year < 2004:
        continue


    four_years_later = final[(final.ano == this_year + 4) & (final.id_municipio == this_mun)]
    eight_years_later = final[(final.ano == this_year + 8) & (final.id_municipio == this_mun)]

    winner_4yr = four_years_later.overall_winner_annie_id

    if len(this_winner) > 1:
        print(i, ' failed!')


    if winner_4yr == this_winner:
        final.loc[eight_years_later.index[0], 'term_limited'] = 1

        

 17%|█▋        | 5671/33479 [00:02<00:11, 2343.65it/s]

5671  failed!


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [199]:
len(this_winner)

14

In [200]:
len(winner_4yr)

1

In [195]:
four_years_later.overall_winner_annie_id


15910    20082102200146301
Name: overall_winner_annie_id, dtype: object

In [191]:
eight_years_later.index[0]

14997

In [182]:
four_years_later.overall_winner_annie_id

10519    20042704302622
Name: overall_winner_annie_id, dtype: object

In [183]:
eight_years_later

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,winner_turno2_sequencial_candidato,winner_special,winner_special_sequencial_candidato,outcomes,margin_special,totalvotes_special,winner,margin,totalvotes,sequencial_candidato
14997,2008,2704302,0.0,1402445,NaN,392495.0,0.0,319831.0,0.0,277883.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1402445,277883.0,392495.0,5663


In [163]:
final.iloc[0]

ano                                                 1996
id_municipio                                     2704302
runoff                                               0.0
winner_turno1                                        NaN
winner_turno2                                        NaN
totalvotes_turno1                                    0.0
totalvotes_turno2                                    0.0
winner_votes_turno_1                                 0.0
winner_votes_turno_2                                 0.0
margin_turno1                                        0.0
margin_turno2                                        0.0
incumbant                                            0.0
term_limited_seat                                    0.0
overall_winner                                       0.0
tipo                                   eleicao ordinaria
special                                              0.0
overall_winner_numero_candidato                      0.0
winner_turno1_sequencial_candid

In [94]:
final_cut = final[(final.ano >= 2004) & (final.ano <= 2016)]

final_cut = final_cut.reset_index(drop = True)



In [ ]:
#find

In [159]:
np.unique(final_cut[final_cut.overall_winner_annie_id.isna()].id_municipio)

array(['2802601', '2803609', '3545506', '5203500'], dtype=object)

In [95]:
final_cut.columns

Index(['ano', 'id_municipio', 'runoff', 'winner_turno1', 'winner_turno2',
       'totalvotes_turno1', 'totalvotes_turno2', 'winner_votes_turno_1',
       'winner_votes_turno_2', 'margin_turno1', 'margin_turno2', 'incumbant',
       'term_limited_seat', 'overall_winner', 'tipo', 'special',
       'overall_winner_numero_candidato', 'winner_turno1_sequencial_candidato',
       'overall_winner_annie_id', 'overall_winner_sequencial_candidato',
       'winner_turno2_sequencial_candidato', 'winner_special',
       'winner_special_sequencial_candidato', 'outcomes', 'margin_special',
       'totalvotes_special', 'winner', 'margin', 'totalvotes',
       'sequencial_candidato'],
      dtype='object')

In [96]:
final_cut.duplicated('annie_id')

KeyError: Index(['annie_id'], dtype='object')

In [ ]:
pd.merge(final_cut, df_turno, how = 'left', left_on = ['ano', 'id_municipio'], right_on = ['ano', 'id_municipio'])

In [53]:
this_ano = 2004
this_muni = '3301009'

this_vote = vote[(vote.ano == this_ano) & (vote.id_municipio == this_muni)]
this_vote1 = vote1[(vote1.ano == this_ano) & (vote1.id_municipio == this_muni)]
this_df = df[(df.ano == this_ano) & (df.id_municipio == this_muni)]

this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
13865,2004,2,eleicao ordinaria,RJ,3301009,58190,prefeito,PDT,12,323,963200,eleito,131363,200433010093232,963200
39225,2004,2,eleicao ordinaria,RJ,3301009,58190,prefeito,PMDB,15,319,378227,nao eleito,109309,200433010093192,378227


In [55]:
final_cut[final_cut.overall_winner_annie_id == '20044215406391'] 

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,...,winner_turno2_sequencial_candidato,winner_special,winner_special_sequencial_candidato,outcomes,margin_special,totalvotes_special,winner,margin,totalvotes,sequencial_candidato
6925,2004,4215406,0.0,1540298,NaN,2148.0,0.0,2148.0,0.0,2148.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1540298,2148.0,2148.0,39
6926,2004,3301009,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,963200,323,"eleito, nao eleito",22054.0,240672.0,NaN,0.0,0.0,NaN


In [97]:
final_cut[final_cut.duplicated('overall_winner_annie_id')][['ano', 'id_municipio', 'overall_winner_annie_id' ]]

,ano,id_municipio,overall_winner_annie_id
9711,2004,2803609,NaN
17830,2012,5203500,NaN
24475,2016,2803609,NaN
26088,2016,3545506,NaN


In [238]:
this_ano = 2004
this_muni = '2802601'

this_vote = vote[(vote.ano == this_ano) & (vote.id_municipio == this_muni)]
this_vote1 = vote1[(vote1.ano == this_ano) & (vote1.id_municipio == this_muni)]
this_df = df[(df.ano == this_ano) & (df.id_municipio == this_muni)]

this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
21205,2004,1,eleicao ordinaria,SE,2802601,31518,prefeito,PT,13,78,516782,nao eleito,11,20042802601781,516782
63220,2004,1,eleicao ordinaria,SE,2802601,31518,prefeito,PFL,25,104,1173870,nao eleito,1771,200428026011041,1173870


In [239]:
this_vote1

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id
21205,2004,1,eleicao ordinaria,SE,2802601,31518,prefeito,PT,13,78,516782,nao eleito,11,20042802601781
63220,2004,1,eleicao ordinaria,SE,2802601,31518,prefeito,PFL,25,104,1173870,nao eleito,1771,200428026011041


In [278]:
winner_ordinary_turno1 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

winner_ordinary_turno1.shape[0]

winner_other_type = vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)&(vote.resultado.isin(['eleito', 'eleito por qp']))]

winner_other_type


,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
29804,2016,1,eleicao suplementar muqui (es),ES,3203809,56758,prefeito,PTB,14,80000013137,679366,eleito,3378,20163203809141,679366


In [ ]:
winner_ordinary_turno1 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

if winner_ordinary_turno1.shape[0] > 1: #more than one possible winner for turno 1
    if '2º turno' in list(winner_ordinary_turno1.resultado): #find the max votes in turno 1
        final.loc[i, 'winner_turno1'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
        final.loc[i, 'winner_turno1_numero_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'numero_candidato']

        #margin for exactly one winner for turno 1.
        number_of_votes = list(this_vote[this_vote.turno == 1].sort_values(by = 'votos', axis = 0, ascending = False).votos)
        if len(number_of_votes) == 1:
            final.loc[i, 'margin_turno1'] = number_of_votes[0] #margin
        else:
            final.loc[i, 'margin_turno1'] = number_of_votes[0] - number_of_votes[1] #margin
        
        final.loc[i, 'winner_votes_turno_1'] = number_of_votes[0] #winner number of votes
        final.loc[i, 'totalvotes_turno1'] = sum(number_of_votes) #total number of votes in turno 1

        did_runoff_occur = 1
    else: 
        error_messages.append("More than one turno 1 winner ordinary election for " + str(this_muni) + " " + str(this_ano))
elif winner_ordinary_turno1.shape[0] == 0:  #missing winner for ordinary election! 
    winner_other_type = vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)&(vote.resultado.isin(['eleito', 'eleito por qp']))]
    if list(winner_other_type.tipo_eleicao):
        final.loc[i, 'special'] = 1
        final.loc[i, 'winner_special'] = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
        final.loc[i, 'winner_special_numero_candidato'] = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'numero_candidato']
        outcome_type = np.unique(vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)].resultado)
        overall_winner = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
        overall_winner_annie_id = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'annie_id']
        overall_winner_numero_candidato = winner_other_type.loc[winner_other_type.votos.idxmax(axis = 'columns'), 'numero_candidato'] #overall_winner_numero_candidato

        final.loc[i, 'overall_winner'] = overall_winner
        final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
        final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato
        
        
        final.loc[i, 'tipo'] = list(winner_other_type.tipo_eleicao)[0]
        final.loc[i, 'outcomes'] = ", ".join(outcome_type)

        number_of_votes = list(vote[(vote.ano == this_ano)&(vote.id_municipio == this_muni)].sort_values(by = 'votos', axis = 0, ascending = False).votos)
        if winner_other_type.shape[0] == 1:
            if len(number_of_votes) == 1:
                final.loc[i, 'margin_special'] = number_of_votes[0] #margin
            else:
                final.loc[i, 'margin_special'] = number_of_votes[0] - number_of_votes[1] #margin
            
            #final.loc[i, 'winner_votes_turno_1'] = number_of_votes[0] #winner number of votes
            final.loc[i, 'totalvotes_special'] = sum(number_of_votes) #total number of votes in turno 1
        else:
            error_messages.append( str(winner_other_type.shape[0])  + 'special election winners for '+ str(this_muni) + " " + str(this_ano ) )
        
    else:
        error_messages.append("No winner turno 1 ordinary election for " + str(this_muni) + " " + str(this_ano ))
        missing_muni.append(this_muni)
        missing_ano.append(this_ano)

    this_vote[(this_vote.tipo_eleicao != 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

else: #this is when there is exactly one winner
    final.loc[i, 'winner_turno1'] = list(winner_ordinary_turno1['id_candidato_bd'])[0]
    final.loc[i, 'winner_turno1_numero_candidato'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'numero_candidato']

    overall_winner = list(winner_ordinary_turno1.id_candidato_bd)[0]
    overall_winner_annie_id = list(winner_ordinary_turno1.annie_id)[0]
    overall_winner_numero_candidato = list(winner_ordinary_turno1.numero_candidato)[0]
    final.loc[i, 'overall_winner'] = overall_winner
    final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
    final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato

    #margin for exactly one winner for turno 1.
    number_of_votes = list(this_vote[this_vote.turno == 1].sort_values(by = 'votos', axis = 0, ascending = False).votos)
    if len(number_of_votes) == 1:
        final.loc[i, 'margin_turno1'] = number_of_votes[0] #margin
    else:
        final.loc[i, 'margin_turno1'] = number_of_votes[0] - number_of_votes[1] #margin
    final.loc[i, 'winner_votes_turno_1'] = number_of_votes[0] #winner number of votes
    final.loc[i, 'totalvotes_turno1'] = sum(number_of_votes) #total number of votes in turno 1


    #if '2º turno' in list(winner_ordinary_turno1.resultado):
    #    print("Not sure if this is supposed to happen:", i, this_ano, this_muni) 
    #   did_runoff_occur = 1


## GET TURNO 2 WINNER AND MARGIN AND TOTAL VOTES

if did_runoff_occur == 1: 
    print(this_muni, this_ano)
    
    winner_ordinary_turno2 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', 'eleito por qp'])) & (this_vote.turno == 2)]

    if winner_ordinary_turno2.shape[0] == 1:
        
        final.loc[i, 'winner_turno2'] = list(winner_ordinary_turno2.id_candidato_bd)[0]
        final.loc[i, 'winner_turno2_numero_candidato'] = list(winner_ordinary_turno2.numero_candidato)[0]

        #use this for incumbant calculation
        overall_winner = list(winner_ordinary_turno2.id_candidato_bd)[0]
        overall_winner_annie_id = list(winner_ordinary_turno2.annie_id)[0]
        overall_winner_numero_candidato = list(winner_ordinary_turno1.numero_candidato)[0]

        final.loc[i, 'overall_winner'] = overall_winner
        final.loc[i, 'overall_winner_annie_id'] = overall_winner_annie_id
        final.loc[i, 'overall_winner_numero_candidato'] = overall_winner_numero_candidato

        #margin for exactly one winner for turno 1.
        number_of_votes_2 = list(this_vote[this_vote.turno == 2].sort_values(by = 'votos', axis = 0, ascending = False).votos)
        if len(number_of_votes_2) == 1:
            final.loc[i, 'margin_turno2'] = number_of_votes_2[0] #margin
        else:
            final.loc[i, 'margin_turno2'] = number_of_votes_2[0] - number_of_votes_2[1] #margin
        final.loc[i, 'winner_votes_turno_2'] = number_of_votes_2[0] #winner number of votes
        final.loc[i, 'totalvotes_turno2'] = sum(number_of_votes_2) #total number of votes in turno 1
    elif winner_ordinary_turno2.shape[0] > 1: 
        error_messages.append('More than one turno 2 winner orginary election for' + str(this_muni) + " " + str(this_ano))
    elif winner_ordinary_turno2.shape[0] == 0 : 
        error_messages.append('No turno 2 winner orginary election for' + str(this_muni) + " " + str(this_ano))


#INCUMBANT
if this_ano >= 2004:
    this_vote_4yr = vote[(vote.ano == this_ano-4)&(vote.id_municipio == this_muni)]

    if overall_winner in list(this_vote_4yr.id_candidato_bd):
        final.loc[i, 'incumbant'] = 1

In [68]:
this_df

,ano,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,id_candidato_bd,cpf,titulo_eleitoral,sequencial,numero,...,instrucao,estado_civil,nacionalidade,sigla_uf_nascimento,municipio_nascimento,email,raca,annie_id,numero_candidato,id_candidato_bd_full
8125,2004,eleicao ordinaria,RS,4302238,89060,1753293,30513383034,061154080434,21,11,...,ensino medio incompleto,casado(a),brasileira,RS,Júlio De Castilhos,NaN,NaN,2004430223811,11,1753293
62232,2004,eleicao ordinaria,RS,4302238,89060,223386,45598088053,025984640400,41,23,...,ensino medio completo,casado(a),brasileira,RS,Ibiruba,NaN,NaN,2004430223823,23,223386


In [80]:
vote = vote.drop_duplicates(['ano', 'id_municipio', 'numero_candidato'])

In [161]:
final_merged = final[(final.ano >= 2004) & (final.ano <= 2016)]

In [162]:
final[final.winner.isna()].groupby(['id_municipio']).size()

id_municipio
1100130    1
1101609    1
1200252    1
1302603    1
1303700    1
          ..
5002001    1
5005202    1
5106273    1
5200605    1
5220454    1
Length: 155, dtype: int64

In [163]:
final[final.winner.isna()].groupby(['ano']).size()

ano
1996      4
2000    148
2004      1
2012      1
2020      1
dtype: int64

In [164]:
vote[vote.id_candidato_bd.isna()].groupby(['ano']).size()

ano
1996    417
2000    185
2004    165
2008     89
2012     76
2016     73
2020     52
dtype: int64

In [152]:
sorted((dict(vote[vote.id_candidato_bd.isna()].groupby(['id_municipio']).size())).items(), key=lambda x:x[1], reverse = True)

[('3550308', 14),
 ('2611606', 13),
 ('2800308', 13),
 ('2704302', 12),
 ('4314902', 12),
 ('2211001', 11),
 ('2304400', 11),
 ('1302603', 10),
 ('1501402', 10),
 ('2111300', 10),
 ('2507507', 10),
 ('2927408', 9),
 ('3205309', 9),
 ('1200401', 8),
 ('3106200', 7),
 ('5002704', 7),
 ('5208707', 7),
 ('5220454', 7),
 ('1721000', 6),
 ('2408102', 6),
 ('1200807', 5),
 ('1400100', 5),
 ('1507755', 5),
 ('1600303', 5),
 ('2206407', 5),
 ('2210102', 5),
 ('2501203', 5),
 ('3169000', 5),
 ('3514809', 5),
 ('4103800', 5),
 ('4113809', 5),
 ('1100106', 4),
 ('1504422', 4),
 ('1504802', 4),
 ('1508084', 4),
 ('2100501', 4),
 ('2100873', 4),
 ('2200459', 4),
 ('2202802', 4),
 ('2203107', 4),
 ('2203305', 4),
 ('2205409', 4),
 ('2206704', 4),
 ('2903508', 4),
 ('2904852', 4),
 ('2910727', 4),
 ('2919504', 4),
 ('3107604', 4),
 ('3126406', 4),
 ('3138708', 4),
 ('3202256', 4),
 ('3503703', 4),
 ('3509452', 4),
 ('3516002', 4),
 ('3523909', 4),
 ('4110607', 4),
 ('4201307', 4),
 ('4322806', 4),
 ('

In [153]:
(dict(vote[vote.id_candidato_bd.isna()].groupby(['id_municipio']).size()))

{'1100056': 2,
 '1100106': 4,
 '1100130': 2,
 '1100205': 1,
 '1100320': 2,
 '1101203': 2,
 '1200138': 3,
 '1200179': 3,
 '1200401': 8,
 '1200807': 5,
 '1301852': 1,
 '1302603': 10,
 '1304005': 1,
 '1400100': 5,
 '1400308': 1,
 '1400506': 2,
 '1501204': 1,
 '1501402': 10,
 '1503101': 2,
 '1503507': 1,
 '1504208': 1,
 '1504307': 1,
 '1504422': 4,
 '1504703': 3,
 '1504802': 4,
 '1505031': 1,
 '1505064': 1,
 '1505494': 3,
 '1506112': 1,
 '1506195': 1,
 '1507003': 1,
 '1507706': 1,
 '1507755': 5,
 '1508001': 3,
 '1508035': 1,
 '1508084': 4,
 '1508357': 1,
 '1600279': 3,
 '1600303': 5,
 '1600501': 1,
 '1701101': 2,
 '1706258': 1,
 '1711951': 1,
 '1712157': 2,
 '1714203': 1,
 '1715101': 2,
 '1715150': 1,
 '1715705': 1,
 '1718451': 1,
 '1719004': 2,
 '1720150': 3,
 '1720903': 1,
 '1721000': 6,
 '2100154': 3,
 '2100204': 2,
 '2100477': 1,
 '2100501': 4,
 '2100873': 4,
 '2101202': 1,
 '2101970': 1,
 '2103307': 1,
 '2103406': 1,
 '2103703': 3,
 '2103752': 1,
 '2104602': 1,
 '2105351': 1,
 '210560

In [154]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388,1996270430241,NaN
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713,1996130260316,NaN
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721,1996292740826,NaN
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705,1996292740829,NaN
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989,1996292740830,NaN


In [155]:
vote[vote.ano == 2000]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos,annie_id,id_candidato_bd_old
27,2000,1,eleicao ordinaria,AL,2700409,27073,prefeito,PGT,30,NaN,1132123,nao eleito,102,2000270040930,1132123
28,2000,1,eleicao ordinaria,AL,2703908,27774,prefeito,PSD,41,NaN,531469,nao eleito,995,2000270390841,531469
29,2000,1,eleicao ordinaria,AL,2705002,27995,prefeito,PSD,41,NaN,1267896,nao eleito,2312,2000270500241,1267896
30,2000,1,eleicao ordinaria,AL,2706448,27006,prefeito,PST,18,NaN,944392,nao eleito,755,2000270644818,944392
31,2000,1,eleicao ordinaria,AL,2707008,28398,prefeito,PST,18,NaN,1440801,nao eleito,329,2000270700818,1440801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101543,2000,1,eleicao ordinaria,SP,3541406,69299,prefeito,PT do B,70,NaN,1103970,nao eleito,1290,2000354140670,1103970
101544,2000,1,eleicao ordinaria,SP,3541604,69337,prefeito,PT do B,70,NaN,595300,nao eleito,73,2000354160470,595300
101545,2000,1,eleicao ordinaria,SP,3548005,70610,prefeito,PT do B,70,NaN,953777,nao eleito,662,2000354800570,953777
101546,2000,1,eleicao ordinaria,SP,3555109,72036,prefeito,PT do B,70,NaN,578266,nao eleito,759,2000355510970,578266


In [ ]:
len(missing_ano)

60

In [ ]:
this_ano = 2016
this_muni = '3539103'

this_vote = vote[(vote.ano == this_ano) & (vote.id_municipio == this_muni)]

In [ ]:
this_vote[(this_vote.tipo_eleicao != 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos


In [ ]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
14759,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PDT,12,250000062374,1681584,nao eleito,235
14814,2016,1,suplementar pirapora do bom jesus,SP,3539103,68837,prefeito,PDT,12,250000094770,1681584,nao eleito,544
29496,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PTB,14,250000056581,1251198,nao eleito,0
41424,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,MDB,15,250000023247,1715925,nao eleito,4227
41611,2016,1,suplementar pirapora do bom jesus,SP,3539103,68837,prefeito,MDB,15,250000094765,1715925,eleito,3861
47386,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PODE,19,250000052651,1239109,nao eleito,291
53096,2016,1,suplementar pirapora do bom jesus,SP,3539103,68837,prefeito,PL,22,250000094773,1510214,nao eleito,1025
67045,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PRTB,28,250000063596,1459709,nao eleito,21
84586,2016,1,suplementar pirapora do bom jesus,SP,3539103,68837,prefeito,PSDB,45,250000094785,1399890,nao eleito,3611
88180,2016,1,eleicao ordinaria,SP,3539103,68837,prefeito,PSOL,50,250000044104,1359826,nao eleito,125


In [ ]:
test_ano = 2016
test_muni = '3539103'

In [ ]:
error_messages

[]

In [ ]:
final[(final.winner_turno1.isna())&(final.ano >= 2002)]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,margin_turno2,incumbant,term_limited_seat,overall_winner
5730,2004,2306801,0.0,NaN,NaN,6170.0,0.0,3155.0,0.0,140.0,0.0,0.0,0.0,NaN
5783,2004,1301100,0.0,NaN,NaN,10437.0,0.0,4547.0,0.0,324.0,0.0,0.0,0.0,NaN
5813,2004,2919504,0.0,NaN,NaN,22643.0,0.0,11663.0,0.0,683.0,0.0,1.0,0.0,NaN
5886,2004,4110607,0.0,NaN,NaN,9771.0,0.0,5732.0,0.0,1693.0,0.0,1.0,0.0,NaN
5940,2004,3126406,0.0,NaN,NaN,1949.0,0.0,1014.0,0.0,443.0,0.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32866,2020,3111002,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32925,2020,3109600,0.0,NaN,NaN,2926.0,0.0,1518.0,0.0,400.0,0.0,1.0,0.0,NaN
33088,2020,2910909,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33228,2020,3304607,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
this_ano = 2004
this_muni = '2919504'

this_vote = vote[(vote.ano == this_ano) & (vote.id_municipio == this_muni)]
winner_ordinary_turno2 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', 'eleito por qp'])) & (this_vote.turno == 2)]

this_vote



,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
36845,2004,1,eleicao ordinaria,BA,2919504,36919,prefeito,PMDB,15,47,NaN,eleito,11663
61947,2004,1,eleicao ordinaria,BA,2919504,36919,prefeito,PFL,25,55,185627,nao eleito,10980


In [ ]:
#TERM LIMIT ON SEAT
    if this_ano >= 2008:
        #check winner 8 years ago with 4 years ago
        winner_8yrs_ago = list(final[(final.ano == this_ano - 8)&(final.id_municipio == this_muni)].overall_winner)
        winner_4yrs_ago = list(final[(final.ano == this_ano - 4)&(final.id_municipio == this_muni)].overall_winner)

        if not winner_8yrs_ago:
            #print("Missing winner for " + this_muni + " " + str(this_ano - 8))
            error_messages.append("Missing winner for " + this_muni + " " + str(this_ano - 8))
        if not winner_4yrs_ago:
            error_messages.append("Missing winner for " + this_muni + " " + str(this_ano - 4))
        if (not not winner_8yrs_ago) and (not not winner_4yrs_ago) and (winner_8yrs_ago == winner_4yrs_ago):
            final.loc[i, 'term_limited_seat'] = 1

'392561'

In [ ]:
final

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,margin_turno2,incumbant,term_limited_seat,overall_winner
0,1996,2200806,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1996,2203107,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1996,2200905,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2000,1200401,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2000,3109600,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2020,1500404,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,2020,4116208,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,2020,3533908,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,2020,4322350,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.head()

,ano,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,id_candidato_bd,cpf,titulo_eleitoral,sequencial,numero,...,data_nascimento,idade,genero,instrucao,estado_civil,nacionalidade,sigla_uf_nascimento,municipio_nascimento,email,raca
0,1996,eleicao ordinaria,PI,2211001,12190,NaN,NaN,NaN,NaN,16,...,1966-11-18,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,eleicao ordinaria,SE,2800308,31054,NaN,NaN,NaN,NaN,16,...,1956-05-23,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,eleicao ordinaria,RS,4314902,88013,NaN,NaN,NaN,NaN,56,...,1950-07-12,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,eleicao ordinaria,MG,3106200,41238,NaN,NaN,NaN,NaN,16,...,1953-06-07,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,eleicao ordinaria,PB,2507507,20516,NaN,NaN,NaN,NaN,16,...,1953-04-04,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
demo_cols = ['id_municipio',
       'id_municipio_tse', 'id_candidato_bd', 'cpf', 'titulo_eleitoral',
       'sequencial', 'numero', 'nome', 'nome_urna', 'numero_partido',
       'sigla_partido', 'cargo', 'situacao', 'ocupacao',
        'idade', 'genero', 'instrucao', 'estado_civil',
       'nacionalidade', 'sigla_uf_nascimento', 'municipio_nascimento',
        'raca']
demo = df[demo_cols]

final_merged = pd.merge(final, demo, how = 'left', left_on = 'overall_winner', right_on = 'id_candidato_bd')

ValueError: You are trying to merge on float64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
final.head()

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,winner_votes_turno_1,winner_votes_turno_2,margin_turno1,margin_turno2,incumbant,term_limited_seat,overall_winner
0,1996,2200806,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1996,2203107,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1996,2200905,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2000,1200401,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2000,3109600,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.head()

,ano,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,id_candidato_bd,cpf,titulo_eleitoral,sequencial,numero,...,data_nascimento,idade,genero,instrucao,estado_civil,nacionalidade,sigla_uf_nascimento,municipio_nascimento,email,raca
0,1996,eleicao ordinaria,PI,2211001,12190,NaN,NaN,NaN,NaN,16,...,1966-11-18,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,eleicao ordinaria,SE,2800308,31054,NaN,NaN,NaN,NaN,16,...,1956-05-23,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,eleicao ordinaria,RS,4314902,88013,NaN,NaN,NaN,NaN,56,...,1950-07-12,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,eleicao ordinaria,MG,3106200,41238,NaN,NaN,NaN,NaN,16,...,1953-06-07,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,eleicao ordinaria,PB,2507507,20516,NaN,NaN,NaN,NaN,16,...,1953-04-04,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
vote[(vote.ano == this_ano - 4)&(vote.id_municipio == this_muni)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
13458,2004,1,eleicao ordinaria,PR,4106571,74241,prefeito,PDT,12,36,392561,eleito,2054


In [ ]:
overall_winner

'392561'

In [ ]:
vote[(vote.ano == this_ano-4)&(vote.id_municipio == this_muni)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
13458,2004,1,eleicao ordinaria,PR,4106571,74241,prefeito,PDT,12,36,392561,eleito,2054


In [ ]:
this_vote_4yr

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
13458,2004,1,eleicao ordinaria,PR,4106571,74241,prefeito,PDT,12,36,392561,eleito,2054


In [ ]:
this_ano = 2000.0
this_muni = '4106571'

this_vote = vote[(vote.ano == this_ano) & (vote.id_municipio == this_muni)]
winner_ordinary_turno2 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', 'eleito por qp'])) & (this_vote.turno == 2)]

number_of_votes = list(winner_ordinary_turno1.sort_values(by = 'votos', axis = 0, ascending = False).votos)



In [ ]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
4057,2000,1,eleicao ordinaria,PR,4106571,74241,prefeito,PPB,11,NaN,1447013,eleito,1767
32799,2000,1,eleicao ordinaria,PR,4106571,74241,prefeito,PMDB,15,NaN,1519996,nao eleito,931


In [ ]:
this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', 'eleito por qp'])) & (this_vote.turno == 2)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
17934,2000,2,eleicao ordinaria,PE,2611606,25313,prefeito,PT,13,NaN,NaN,eleito por qp,382988


In [ ]:
number_of_votes_2 = list(winner_ordinary_turno2.sort_values(by = 'votos', axis = 0, ascending = False).votos)

In [ ]:
number_of_votes_2

[382988]

In [ ]:
list(winner_ordinary_turno2.votos)[0]

382988

In [ ]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
60236,2000,1,eleicao ordinaria,SP,3502606,61514,prefeito,PFL,25,NaN,252375,eleito,2657


In [ ]:
number_of_votes = list(this_vote.sort_values(by = 'votos', axis = 0, ascending = False).votos)
this_margin = number_of_votes[0] - number_of_votes[1]

[5368, 5019, 3970, 435, 117]

In [ ]:
winner_ordinary_turno1

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
86495,2020,1,eleicao ordinaria,MG,3111606,42315,prefeito,PSDB,45,130001000415,1375145,eleito,5368


In [ ]:
this_ano = 2012
this_muni = '1302603'

this_vote = vote[(vote.ano == this_ano) & (vote.id_municipio == this_muni)]

#winner for ordinary election turno 1
winner_ordinary_turno1 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

if winner_ordinary_turno1.shape[0] > 1: 
    if '2º turno' in list(winner_ordinary_turno1.resultado):
        final.loc[i, 'winner_turno1'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
    else:  
        write_error("More than one turno 1 winner ordinary election for " + str(this_muni) + " " + str(this_ano ))
elif winner_ordinary_turno1.shape[0] == 0: 
    write_error("No winner turno 1 ordinary election for " + str(this_muni) + " " + str(this_ano ))
else:
    #final.loc[i, 'winner_turno1'] = winner_ordinary_turno1.id_candidato_bd
    final.loc[i, 'winner_turno1'] = list(winner_ordinary_turno1['id_candidato_bd'])[0]

YES


In [ ]:
'2º turno' in list(winner_ordinary_turno1.resultado)

True

In [ ]:
winner_ordinary_turno1

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
84660,2012,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSDB,45,40000003089,1463217,2º turno,385855
94227,2012,1,eleicao ordinaria,AM,1302603,2550,prefeito,PC do B,65,40000005359,1476233,2º turno,189861


In [ ]:
final[final.ano == 2012]

,ano,id_municipio,runoff,winner_turno1,winner_turno2,totalvotes_turno1,totalvotes_turno2,margin_turno1,margin_turno2,incumbant,term_limited_seat
1059,2012,2704302,0.0,1331712,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1060,2012,2705200,0.0,153735,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1061,2012,2709400,0.0,264699,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1062,2012,1302603,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1063,2012,1303106,0.0,1539043,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
33402,2012,2513851,0.0,716101,NaN,0.0,0.0,0.0,0.0,0.0,0.0
33403,2012,1100155,0.0,1555761,NaN,0.0,0.0,0.0,0.0,0.0,0.0
33404,2012,3525409,0.0,877066,NaN,0.0,0.0,0.0,0.0,0.0,0.0
33405,2012,1708254,0.0,650843,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
this_vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
842,2016,1,eleicao ordinaria,AP,1600303,6050,prefeito,PSTU,16,30000001275,648417,nao eleito,11242
843,2016,1,eleicao ordinaria,AP,1600303,6050,prefeito,REDE,18,30000002139,31551,2º turno,95129
1215,2016,2,eleicao ordinaria,AP,1600303,6050,prefeito,REDE,18,30000002139,31551,eleito,123808
1923,2016,1,eleicao ordinaria,AP,1600303,6050,prefeito,REPUBLICANOS,10,30000001715,302084,nao eleito,25365
21617,2016,1,eleicao ordinaria,AP,1600303,6050,prefeito,PT,13,30000001932,115779,nao eleito,2174
39362,2016,1,eleicao ordinaria,AP,1600303,6050,prefeito,MDB,15,30000001381,1374364,2º turno,56256
41615,2016,2,eleicao ordinaria,AP,1600303,6050,prefeito,MDB,15,30000001381,1374364,nao eleito,80840
71553,2016,1,eleicao ordinaria,AP,1600303,6050,prefeito,PSB,40,30000002368,1274303,nao eleito,7922
88928,2016,1,eleicao ordinaria,AP,1600303,6050,prefeito,PATRIOTA,51,30000001914,152074,nao eleito,15271


In [ ]:
winner_ordinary_turno1 = this_vote[(this_vote.tipo_eleicao == 'eleicao ordinaria') & (this_vote.resultado.isin(['eleito', '2º turno'])) & (this_vote.turno == 1)]

if winner_ordinary_turno1.shape[0] > 1: 
    if '2º turno' in winner_ordinary_turno1.resultado:
        final.loc[i, 'winner_turno1'] = winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'id_candidato_bd']
    else:  
        write_error("More than one turno 1 winner ordinary election for " + str(this_muni) + " " + str(this_ano ))
elif winner_ordinary_turno1.shape[0] == 0: 
    write_error("No winner turno 1 ordinary election for " + str(this_muni) + " " + str(this_ano ))
else:
    final.loc[i, 'winner_turno1'] = winner_ordinary_turno1.id_candidato_bd

    

In [ ]:
winner_ordinary_turno1.loc[winner_ordinary_turno1.votos.idxmax(axis = 'columns'), 'id_candidato_bd']

'31551'

In [ ]:
winner_ordinary_turno1.votos.idxmax(axis = 'columns')

843

In [ ]:
with open('errors.txt', 'w') as f:
    f.write('Create a new text file!')